![title](./HeaderImg.png)

# MariaDB AX ColumnStore: Built-In Statistical Functions

## Introduction

Business Intelligence (BI) and Data Science (DS) require processing huge amounts of data in various and complex ways using a vast array of statistical methods and tools. MariaDB AX lets you scale analytic workloads and analyze data in unforeseen ways without having to create query-optimized schemas and indexes. Furthermore, MariaDB AX ColumnStore interoperates with modern BI and DS tools. 

For data engineers, data analysts and data scientists, having an interactive computing environment that can drive exploration using notebooks (e.g., Jupyter) and structured query language (SQL) enables reliable iterative conversation between researcher and data. An example of MariaDB AX ColumnStore support for statistical functions within a Jupyter notebook is presented here.  

## MariaDB AX ColumnStore

MariaDB AX ColumnStore is a columnar database for modern, on-demand analytics. It uses distributed storage and massively parallel processing (MPP) to execute interactive, ad hoc queries on hundreds of terabytes of near-real-time data, with standard SQL and without creating indexes. MariaDB AX scales on commodity hardware, on premises or in the cloud, eliminating the need to spend budget on proprietary data warehouses and appliances.

![title](./Columnar_Dist_Data_Storage.png)

Column oriented databases (a.k.a. columnar databases) are more suitable for analytical workloads because the data format lends itself to faster query processing. These database systems have been shown to perform more than an order of magnitude better than traditional row-oriented database systems.

## Jupyter Notebook and MariaDB

Jupyter notebooks success is it excels in a form of programming called __[“literate programming”](https://en.wikipedia.org/wiki/Literate_programming/)__. Literate programming allows users to formulate and describe their thoughts with prose, supplemented by mathematical equations, as they prepare to write code blocks.

The MariaDB ColumnStore engine is extremely flexible. When you create a table on MariaDB ColumnStore, the system creates at least one file per column in the table. Thus, in a columnar storage format, the query touches only few columns and it is faster to load all values of a “particular column” into memory from disk in fewer Input/Output (I/O) operations and further into CPU cache in fewer instructions. Visit the MariaDB __[ColumnStore Storage Architecture](https://mariadb.com/kb/en/library/columnstore-storage-architecture/)__ page to get the details of the engine. 

Now, let’s see how all this fit together with an example. Start by creating a Jupyter notebook. The version of the notebook server in this example is: 5.6.0 using Python 3.7.0. The MariaDB AX Server version is 10.3.11 using the Columnstore engine version 1.2.2-1. 

The __pymsql__ module is used to connect from the Jupyter notebook to MariaDB. Import the module first, just as you would any other module. Then establish a database connection providing user credentials and server information. 

In [7]:
from IPython.display import HTML, display, Latex       # Used to display data as HTML tables
import pandas as pd                                    # Used to simplify displaying of the result set from SQL statements
import pymysql as MariaDB                              # Used to connect to MariaDB ColumnStore
import warnings                                        # Used for printing warning messages

# Connection credentials and server information
conn = MariaDB.connect( host = '172.17.0.2'     
                     , port = 3306
                     , user = 'mdb'
                     , passwd = 'letmein'
                     , db = 'test'
                     , charset = 'utf8')
cur = conn.cursor()                                    # Create a database cursor to process the result set

For this example, we are going to use three tables with few rows. Table __employees__ and __breeds__ have data about employees and dog breeds respectively. Table __stats__ is used to demonstrate basic statistical functions. It holds the result of the mean, variance and standard deviation calculation of the different dog breeds. Employees data is used to demonstrate the advanced statistical functions such as co-variance and correlation.

| <p>Table stats<p> | <p>Table breeds</p> | <p>Table employees</p> |
| --- | --- | --- |
|<p>CREATE TABLE stats ( <br>&emsp;Mean decimal(12,6), <br>&emsp;Variance double(22,6), <br>&emsp;Std_Dev double(22,6)<br>) <b>ENGINE=InnoDB</b> <br>DEFAULT CHARSET=latin1</p>| <p>CREATE TABLE breeds ( <br>&emsp;id int(11), <br>&emsp;breed varchar(125), <br>&emsp;min_height decimal(8,2), <br>&emsp;max_height decimal(8,2), <br>&emsp;avg_height decimal(8,2), <br>&emsp;min_weight decimal(8,2), <br>&emsp;max_weight decimal(8,2), <br>&emsp;avg_weight decimal(8,2)<br>) <b>ENGINE=Columnstore</b> <br>DEFAULT CHARSET=latin1 </p>| <p>CREATE TABLE employees ( <br>&emsp;employee_id decimal(10,0), <br>&emsp;first_name varchar(20), <br>&emsp;last_name varchar(25),<br>&emsp;email varchar(25), <br>&emsp;phone_number varchar(20)<br>&emsp;hire_date date, <br>&emsp;job_id varchar(10) ,<br>&emsp;salary decimal(8,2), <br>&emsp;commission_pct decimal(2,2), <br>&emsp;manager_id decimal(10,0), <br>&emsp;department_id decimal(10,0)<br>) <b>ENGINE=Columnstore</b> <br>DEFAULT CHARSET=latin1</p>|

## Basic Statistical Functions

MariaDB AX ColumStore comes with a number of aggregate statistical functions used to process complex data sets.

### Mean, Variance and Standard Deviation

You have just got data with data about dog breeds. Each row describes one breed with attributes such as the height and weight of the breed in inches and pounds. Using basic statistics, you need to determine what is the __"normal"__ height and what is __"extra large"__ or __"extra small"__ dog breeds. For the purpose of this example an extra-large or extra-small breed is defined as one standard deviation from the average height mean. The code snippet below can be used to calculate the mean, variance and the "Population Standard Deviation".

In [2]:
try:
    cur.execute("truncate table stats")                       # Delete all the data 
except Exception as e:
    print ( "MySQL Error [%d]: %s" % (e.args[0], e.args[1]) )

# Calculate the mean, variance and standard deviation
sql = '''INSERT INTO stats 
                SELECT AVG( avg_height )
                     , VARIANCE( avg_height )
                     , STDDEV_POP( avg_height )
             FROM breeds
    '''

try:
    cur.execute(sql)                                           # Execute query
except Exception as e:
    print ( "MySQL Error [%d]: %s" % (e.args[0], e.args[1]) )
    
conn.commit()                                                  # Commit insert

sql = '''SELECT *
         FROM stats
    '''

try:
    cur.execute(sql)
except Exception as e:
    print ( "MySQL Error [%d]: %s" % (e.args[0], e.args[1]) )

df = pd.read_sql(sql,conn)
display( df )                                                  # Display the result set

,Mean,Variance,Std_Dev
0,16.35625,43.162148,6.569791


Using the calculations above we can now show the dog breeds that are within one standard deviation (6.5 in.) of the mean (16.3 in.).  

In [3]:
# Find all dog breeds that one standard deviation from the mean.
sql = '''SELECT breed AS Breed
              , (SELECT Mean FROM stats) AS Mean
              , (SELECT Std_dev FROM stats) AS "Std_Dev"
              , avg_height 
         FROM test.breeds
         WHERE avg_height > (SELECT mean + Std_Dev FROM stats)
            OR avg_height < (SELECT mean - Std_Dev FROM stats)
         ORDER BY avg_height
'''

try:
    cur.execute(sql)
except Exception as e:
    print ( "MySQL Error [%d]: %s" % (e.args[0], e.args[1]) )
    
df = pd.read_sql(sql,conn)
# Create a derived column to identify the breed as Extra-Large or Extra-Small
df['Deviation from Mean'] = df.apply(lambda row: 'Extra large' if row.avg_height>row.Mean+row.Std_Dev 
                                                               else 'Extra small', axis=1)
display( df )

,Breed,Mean,Std_Dev,avg_height,Deviation from Mean
0,Chihuahua,16.35625,6.569791,7.5,Extra small
1,Yorkshire Terrier,16.35625,6.569791,8.5,Extra small
2,Maltese,16.35625,6.569791,9.0,Extra small
3,Silky Terrier,16.35625,6.569791,9.5,Extra small
4,Papillion,16.35625,6.569791,9.5,Extra small
5,Pomeranian,16.35625,6.569791,9.5,Extra small
6,Shih Tzu,16.35625,6.569791,9.5,Extra small
7,Toy Poodle,16.35625,6.569791,9.5,Extra small
8,Boxer,16.35625,6.569791,23.0,Extra large
9,Labrador Retriever,16.35625,6.569791,23.0,Extra large


From the output above we can see which breeds are, on average, one standard deviation below or above the mean.

## Advanced Statitical Functions

Covariance and correlation were added to MariaDB in version 1.2.2-1. These are two mathematical concepts which are quite commonly used in business analytics. Both are used to determine the relationship and dependency measure between two random variables.

### Covariance

In this example, the COVAR_POP aggregate function is used to calculate the population covariance of the full employee data set (use COVAR_SAMP for sample data). The SQL statement below is used to show how two variables, __hire_date__ and __salary__, are related.

In [4]:
# Calculate the covariance between the number of days since the employee was hired and his/her salary for 
# department ids 50 and 80
sql='''
   SELECT job_id AS "Job Id"
        , COVAR_POP ( DATEDIFF ( SYSDATE(), hire_date ), salary) AS "Covariance Population"
   FROM employees
   WHERE department_id in (50, 80)
   GROUP BY job_id
'''

try:
    cur.execute(sql)
except Exception as e:
    print("MySQL Error [%d]: %s"%(e.args[0],e.args[1]))

df = pd.read_sql(sql,conn)
display( df )                                            # Display the result set

,Job Id,Covariance Population
0,SA_MAN,660700.000000
1,ST_MAN,436092.000000
2,SA_REP,579988.466112
3,SH_CLERK,212432.500000
4,ST_CLERK,176577.250000


The output shows that hire_date and salary attributes are linearly associated. A positive covariance means the variables are positively related, that is, as the number of days since the employee was hired increases the salary increases as well. 

### Correlation

The CORR(ColumnY, ColumnX) function can be used to find the correlation coefficient to identify the strength of the linear association between the __hire_data__ and __salary__ attributes.

In [13]:
# Getting correlation between hire date and salary for each job id in deparetments 50 and 80
sql = '''
    SELECT employee_id AS "Employee Id"
         , job_id AS "Job Id"
         , salary AS "Salary"
         , hire_date AS "Hire Date"
         , DATEDIFF (SYSDATE(), hire_date) / 365.25 as "Years Employed"
         , CORR( DATEDIFF (SYSDATE(), hire_date), salary ) OVER ( PARTITION BY job_id ) AS "Correlation"
    FROM employees
    WHERE department_id in (50, 80)
    ORDER BY 2, 1;
'''

try:
    cur.execute(sql)
except Exception as e:
    print ( "MySQL Error [%d]: %s" % (e.args[0], e.args[1]) )
       
df = pd.read_sql(sql,conn)
display (df.head(10))                                            # Display first 10 tuples of the result set   

,Employee Id,Job Id,Salary,Hire Date,Years Employed,Correlation
0,145.0,SA_MAN,14000.0,1996-10-01,22.2368,0.912386
1,146.0,SA_MAN,13500.0,1997-01-05,21.9740,0.912386
2,147.0,SA_MAN,12000.0,1997-03-10,21.7988,0.912386
3,148.0,SA_MAN,11000.0,1999-10-15,19.2005,0.912386
4,149.0,SA_MAN,10500.0,2000-01-29,18.9103,0.912386
5,150.0,SA_REP,10000.0,1997-01-30,21.9055,0.804368
6,151.0,SA_REP,9500.0,1997-03-24,21.7604,0.804368
7,152.0,SA_REP,9000.0,1997-08-20,21.3525,0.804368
8,153.0,SA_REP,8000.0,1998-03-30,20.7447,0.804368
9,154.0,SA_REP,7500.0,1998-12-09,20.0493,0.804368


Finally, to terminate the cursor and connection to the database execute the following two lines of code.

In [6]:
cur.close()
conn.close()

## Conclusion

In this blog Jupyter notebook was used to demonstrate new window and statistical aggregate capabilities of MariaDB AX ColumStore. Some key takeaways are:

* Columnar Storage - Column oriented databases (a.k.a., columnar databases) are more suitable for analytical workloads because the data format (column format) lends itself to faster query processing — scans, aggregation et cetera.
* Data discovery - Enable data engineers, data analysts and data scientists, to have an interactive computing environment that can drive exploration using notebooks (e.g., Jupyter) and structured query language (SQL).
* Easy to maintain - Database Administrators love the fact that MariaDB AX Columnstore does not add complexity to the analytical and exploratory environment.
* SQL front end - Data Scientist has been using SQL for their whole career.  No retraining is needed.

Finally, MariaDB is moving to provide an integrated environmet for transactional and analytical workloads that is intuitive yet powerful, cost-effective way to integrate your transactional databases and analytical data stores in a single interface that can be access from modern tools .


## References

MariaDB Origins and versions
<br>https://searchdatamanagement.techtarget.com/definition/MariaDB
<br>ColumnStore Distributed Aggregate Functions
<br>https://mariadb.com/kb/en/library/columnstore-distributed-aggregate-functions/
<br>ColumnStore Architecture
<br>https://mariadb.com/kb/en/library/columnstore-architecture/
<br>ColumnStore SQL Structure and Commands
<br>https://mariadb.com/kb/en/library/columnstore-sql-structure-and-commands/
<br>Python pymysql module 
<br>https://pypi.org/project/PyMySQL/